<font color='red'> Penalty kick application based on 'Professionals Play Minimax' by Ignacio Palacios-Huerta (2003). </font>

### Make Necessary Imports

In [ ]:
from bokeh.io import show, output_notebook, save, output_file
import penaltykickgame as game

### Configure Bokeh For Use In Jupyter Notebook

In [ ]:
output_notebook()

### Make Game

In [ ]:
grid1 = game.main_game.MainGame().make_game()

### Show Game

In [ ]:
show(grid1)

### Create HTML File

In [ ]:
output_file("penalty_kick_game.html")
save(grid1)